In [1]:
from gensim.models import Word2Vec

In [10]:
docs = ['you say goodbye and I say hello .']

sentences = [list(sentence.split(' ')) for sentence in docs]

model = Word2Vec(size = 3, window = 1, min_count = 1, sg = 1) # sg == 0 : CBOW, sg == 1 : Skip-gram
model.build_vocab(sentences)
model.wv.most_similar('say')

[('and', -0.058282725512981415),
 ('goodbye', -0.24626266956329346),
 ('hello', -0.579779863357544),
 ('.', -0.7342768907546997),
 ('you', -0.9281750321388245),
 ('I', -0.9432371258735657)]

In [13]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import os

In [14]:
os.getcwd()

'c:\\Users\\EonKim\\Desktop\\github\\DL\\NLP\\Word2Vec\\Skip-gram'

In [39]:
movie = pd.read_csv('dataset\\ratings_small.csv')
movie.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [40]:
movie = movie.sort_values(by = 'timestamp', ascending = True).reset_index(drop = True)
movie.head()

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,21,5.0,828212412
4,409,25,4.0,828212412


In [41]:
meta = pd.read_csv('dataset\\movies_metadata.csv', low_memory=False)
meta.shape

(45466, 24)

In [42]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [43]:
meta = meta.rename(columns = {'id' : 'movieId'})
movie['movieId'] = movie['movieId'].astype('str')
meta['movieId'] = meta['movieId'].astype('str')

movie = pd.merge(movie, meta[['movieId', 'original_title']], how = 'left', on = 'movieId')
movie.head()

,userId,movieId,rating,timestamp,original_title
0,383,21,3.0,789652009,The Endless Summer
1,383,47,5.0,789652009,NaN
2,383,1079,3.0,789652009,NaN
3,409,21,5.0,828212412,The Endless Summer
4,409,25,4.0,828212412,Jarhead


In [44]:
# UserId 별 MovieIDd 구매 목록 생성
item_agg = movie.groupby(['original_title'])['userId'].agg({'nunique'}).reset_index()
item_agg = item_agg[item_agg['nunique'] >= 5]['original_title'].values 
item_agg # 영화 제목들을 뽑아내는 과정.

array(['...Più forte ragazzi!', '10 Items or Less',
       '10 Things I Hate About You', ..., '사마리아', '해안선', '활'],
      dtype=object)

In [45]:
user_agg = movie.groupby(['userId'])['original_title'].agg({'nunique'}).reset_index()
user_agg = user_agg[user_agg['nunique'] >= 5]['userId'].values
user_agg[:5] # user value 

array([1, 2, 3, 4, 5], dtype=int64)

In [53]:
# 5회 미만 구매된 상품, 5회 미만 구매한 고객은 제거.

movie['check1'] = 0
movie.loc[movie['original_title'].isin(item_agg), 'check1'] = 1

movie['check2'] = 0
movie.loc[movie['userId'].isin(user_agg), 'check2'] = 1

movie = movie.loc[(movie['check1'] == 1) & (movie['check2'] == 1)].reset_index(drop = True)

In [54]:
movie = movie[movie['original_title'].notnull()].reset_index(drop = True)
movie


,userId,movieId,rating,timestamp,original_title,check1,check2
0,383,21,3.0,789652009,The Endless Summer,1,1
1,409,21,5.0,828212412,The Endless Summer,1,1
2,409,25,4.0,828212412,Jarhead,1,1
3,409,30,5.0,828212412,彼女の想いで,1,1
4,409,35,4.0,828212412,The Simpsons Movie,1,1
...,...,...,...,...,...,...,...
42275,251,2762,5.0,1476552557,Young and Innocent,1,1
42276,251,40819,4.0,1476555871,Best Worst Movie,1,1
42277,251,1265,4.0,1476622980,Bridge to Terabithia,1,1
42278,251,44191,3.0,1476623070,Loose Screws,1,1


In [59]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."


In [66]:
movie['original_title'].unique()

array(['The Endless Summer', 'Jarhead', '彼女の想いで', ..., 'Stitches',
       'Wuthering Heights', 'Rise of the Zombies'], dtype=object)

Word2Vec

In [71]:
sentence = []
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [74]:
from gensim.models import Word2Vec 

In [75]:
embedding_model = Word2Vec(sentence, size = 20, window = 5, min_count = 1, workers = 4, iter = 200, sg = 1)

In [76]:
embedding_model.wv.most_similar(positive = ['Spider-Man 2'], topn = 10)

[('Forrest Gump', 0.8018356561660767),
 ('Snow Cake', 0.7177414894104004),
 ('Sunrise: A Song of Two Humans', 0.7063199281692505),
 ('Domicile Conjugal', 0.701453447341919),
 ('Some Like It Hot', 0.6891652941703796),
 ('Tillsammans', 0.679341733455658),
 ('Fail-Safe', 0.6674575209617615),
 ('Krull', 0.6566325426101685),
 ('Conquest of the Planet of the Apes', 0.648646891117096),
 ('Me, Myself & Irene', 0.6426188945770264)]

Doc2Vec

In [77]:
from gensim.models import doc2vec 

In [79]:
meta = pd.read_csv('dataset\\movies_metadata.csv', low_memory=False)
meta = meta[meta['original_title'].notnull()].reset_index(drop = True)
meta = meta[meta['overview'].notnull()].reset_index(drop = True)
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [2]:
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize 
import re 
stop_words = set(stopwords.words('english'))

In [92]:
overview = []
for words in tqdm(meta['overview']):
    word_tokens = word_tokenize(words)
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    sentence = sentence.strip()

    sentence_tokens = word_tokenize(sentence)

    result = ''
    for token in sentence_tokens:
        if token not in stop_words:
            result += ' ' + token
    result = result.strip().lower()
    overview.append(result)


In [95]:
meta['pre_overview'] = overview 

In [96]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm = 0,            # PV-DBOW / default = 1
    dbow_words = 1,    # w2v simultaneous with DBOW d2v / default 0
    window = 10,       # distance between the predicted word and context words
    size = 100,        # vector size 
    alpha = 0.025,     # learning-rate
    seed = 1234,       
    min_count = 5,     # ignore with freq lower
    min_alpha = 0.025, # min learning-rate
    workers = 4 ,      # multi cpu
    hs = 1,            # hierar chical softmax / default 0
    negative = 10      # negative sampling / default 5 
)

C:\Users\EonKim\anaconda3\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [100]:
from collections import namedtuple

agg = meta[['id', 'original_title', 'pre_overview']]
TaggendDocument = namedtuple('TaggendDocument', 'words tags')
taggend_train_docs = [TaggendDocument((c), [d]) for d, c in agg[['original_title', 'pre_overview']].values]

In [101]:
doc_vectorizer.build_vocab(taggend_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d100,n10,hs,w10,mc5,s0.001,t4)


In [1]:
from time import time

start = time()

for epoch in tqdm(range(5)):
    doc_vectorizer.train(taggend_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

#doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
end = time()
print("During Time: {}".format(end-start))

NameError: name 'tqdm' is not defined

In [ ]:
doc_vectorizer.docvecs.most_similar('Toy Story', topn=20)